In [3]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
"""
Aux functions
"""

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

'\nAux functions\n'

In [5]:
"""
Reading dataframes

"""

INPUT_DIR_PATH = ''
DAYS_PRED = 28
DATASET_SIZE = 1947
TR_LAST = DATASET_SIZE - 28 - 28
VL_LAST = DATASET_SIZE - 28
TS_LAST = DATASET_SIZE

def read_data():
    sell_prices_df = pd.read_csv(INPUT_DIR_PATH + 'sell_prices.csv')
    sell_prices_df = reduce_mem_usage(sell_prices_df)
    print('Sell prices has {} rows and {} columns'.format(sell_prices_df.shape[0], sell_prices_df.shape[1]))

    calendar_df = pd.read_csv(INPUT_DIR_PATH + 'calendar.csv')
    calendar_df = reduce_mem_usage(calendar_df)
    calendar_df = calendar_df.fillna('unknown')
    print('Calendar has {} rows and {} columns'.format(calendar_df.shape[0], calendar_df.shape[1]))

    sales_df = pd.read_csv(INPUT_DIR_PATH + 'sales_train_evaluation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_df.shape[0], sales_df.shape[1]))

    submission_df = pd.read_csv(INPUT_DIR_PATH + 'sample_submission.csv')
    return sell_prices_df, calendar_df, sales_df, submission_df
    
prices_df, calendar_df, sales_df, submission_df = read_data()

num_cols = [f"d_{day}" for day in range(0,TR_LAST+1)]
cat_cols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']

df = pd.melt(sales_df,
                  id_vars = cat_cols,
                  value_vars = [col for col in sales_df.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")

df = df.merge(calendar_df, on= "d", copy = False)
df = df.merge(prices_df, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
del sales_df, calendar_df, prices_df
# gc.collect()

df_raw = df.copy()

'\nReading dataframes\n\n'

Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Sales train validation has 30490 rows and 1947 columns


In [12]:
df = df_raw
df = df.rename(columns={'sales':'y', 'date':'ds'})
df.set_index('ds')

,id,item_id,dept_id,store_id,cat_id,state_id,d,y,wm_yr_wk,weekday,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
ds,,,,,,,,,,,,,,,,,,,,,
2011-01-29,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,CA_1,HOBBIES,CA,d_1,12,11101,Saturday,...,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.459961
2011-01-30,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,CA_1,HOBBIES,CA,d_2,15,11101,Sunday,...,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.459961
2011-01-31,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,CA_1,HOBBIES,CA,d_3,0,11101,Monday,...,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.459961
2011-02-01,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,CA_1,HOBBIES,CA,d_4,0,11101,Tuesday,...,2,2011,unknown,unknown,unknown,unknown,1,1,0,0.459961
2011-02-02,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,CA_1,HOBBIES,CA,d_5,0,11101,Wednesday,...,2,2011,unknown,unknown,unknown,unknown,1,0,1,0.459961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-05-22,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,WI_3,FOODS,WI,d_1941,2,11617,Sunday,...,5,2016,unknown,unknown,unknown,unknown,0,0,0,3.980469
2016-05-21,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,WI_3,FOODS,WI,d_1940,1,11617,Saturday,...,5,2016,unknown,unknown,unknown,unknown,0,0,0,1.280273
2016-05-22,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,WI_3,FOODS,WI,d_1941,0,11617,Sunday,...,5,2016,unknown,unknown,unknown,unknown,0,0,0,1.280273


In [13]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,y,ds,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,CA_1,HOBBIES,CA,d_1,12,2011-01-29,11101,...,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.459961
1,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,CA_1,HOBBIES,CA,d_2,15,2011-01-30,11101,...,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.459961
2,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,CA_1,HOBBIES,CA,d_3,0,2011-01-31,11101,...,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.459961
3,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,CA_1,HOBBIES,CA,d_4,0,2011-02-01,11101,...,2,2011,unknown,unknown,unknown,unknown,1,1,0,0.459961
4,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,CA_1,HOBBIES,CA,d_5,0,2011-02-02,11101,...,2,2011,unknown,unknown,unknown,unknown,1,0,1,0.459961


Checking if the wave is stationary 

In [ ]:
from statsmodels.tsa.stattools import adfuller

adf, pvalue, usedlag_, nobs_, critical_values_, icbest_ = adfuller(df['y'])

print("")
print(f'adf (Test Statistic) is: {adf}')
print(f'p-value is: {pvalue}')
print(f'usedlag_ is: {usedlag_}')
print(f'nobs_ is: {nobs_}')
print(f'critical_values_ is: {critical_values_}')
print(f'icbest_ is: {icbest_}')



res = sm.tsa.adfuller(df['y'].dropna(),regression='ct')
print('p-value:{}'.format(res[1]))

res = sm.tsa.adfuller(df['y'].diff().dropna(),regression='c')
print('p-value:{}'.format(res[1]))

In [ ]:
df = df['y'].reset_index()
df

In [ ]:
m = Prophet()
m.fit(df)

future= m.make_future_dataframe(periods=28)

forecast=m.predict(future)
fig1 =m.plot(forecast, figsize=(20,5))

In [ ]:
fig1 = m.plot_components(forecast)

In [ ]:
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

In [ ]:
m.changepoints

In [ ]:
#The magnitude of the changepoints of model m

deltas = m.params['delta'].mean(0)
fig = plt.figure(facecolor='w', figsize=(10, 6))
ax = fig.add_subplot(111)
ax.bar(range(len(deltas)), deltas, facecolor='#0072B2', edgecolor='#0072B2')
ax.grid(True, which='major', c='gray', ls='-', lw=1, alpha=0.2)
ax.set_ylabel('Rate change')
ax.set_xlabel('Potential changepoint')
fig.tight_layout();

In [ ]:
m2 = Prophet(changepoint_range=0.9)
forecast = m2.fit(train_dataset).predict(future)
fig= m2.plot(forecast);
a = add_changepoints_to_plot(fig.gca(), m2, forecast)

In [ ]:
# Adjusting the trend. Default changepoint_prior_scale is 0.05; increase to get more flex on trend

m3 = Prophet(n_changepoints=30, yearly_seasonality=True, changepoint_prior_scale=0.20)
forecast = m3.fit(train_dataset).predict(future)
fig= m3.plot(forecast);
a = add_changepoints_to_plot(fig.gca(), m3, forecast)

In [ ]:
# Adding holidays or events that can affect the time series values.
# Use windows to extend the effect to previous (-) or next (+) days.

avocado_season = pd.DataFrame({
  'holiday': 'avocado season',
  'ds': pd.to_datetime(['2014-07-31', '2014-09-16', 
                        '2015-07-31', '2015-09-16',
                        '2016-07-31', '2016-09-16',
                        '2017-07-31', '2017-09-16',
                       '2018-07-31', '2018-09-16',
                        '2019-07-31', '2019-09-16']),
  'lower_window': -1,
  'upper_window': 0,
})



In [ ]:
m4= Prophet(n_changepoints=20, yearly_seasonality=True, changepoint_prior_scale=0.08, holidays=avocado_season)
m4.fit(train_dataset)
future_data = m4.make_future_dataframe(periods=28, freq = 'd')
 
#forecast the data for future data
forecast_data = m4.predict(future_data)
m4.plot(forecast_data);

In [ ]:
# Adding multiple regressors

train_dataset['wday'] = df_input['wday']
train_X= train_dataset[:1500]
test_X= train_dataset[1500:]

In [ ]:
#Additional Regressor
m5= Prophet(n_changepoints=20, yearly_seasonality=True, changepoint_prior_scale=0.08, holidays=avocado_season)
m5.add_regressor('wday')

#Fitting the data
m5.fit(train_X)
future_data = m5.make_future_dataframe(periods=28)

#forecast the data for Test  data
forecast_data = m5.predict(test_X)
m5.plot(forecast_data);

In [ ]:
df_cv = cross_validation(m, initial='1500 days', period='180 days', horizon = '365 days')
df_cv.head()

In [ ]:
df_p = performance_metrics(df_cv)
df_p.tail()

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='mae')